In [26]:
import pandas as pd
import pyomo.environ as pyo
from idaes.core.solvers import get_solver


In [27]:
data_path_syensqo = 'General_dataset_DEHPA.xlsx'

df_syensqo = pd.read_excel(data_path_syensqo, sheet_name="Syensqo")

In [28]:
da_silva_param = 'da_silva_parameters.xlsx'
df_da_silva_param = pd.read_excel(da_silva_param)
df_da_silva_param = df_da_silva_param.set_index(df_da_silva_param.columns[0])
df_da_silva_param

,a1i,a2i,c1i,c2i,c3i,c4i,d1i,d2i,d3i,b1i,b2i,f
Unnamed: 0,,,,,,,,,,,,
Mn,1.287143,1.137745,-10.336027,-871.782027,-7568.218054,2949.020375,29.724648,3.597103e+07,-1.021938e+07,-1.913048,-0.089758,0.654000
Co,0.893839,0.324978,6.795728,-2123.213776,-16857.747213,2275.249687,-45.500572,8.160990e+07,-1.058155e+07,-1.729339,-0.036104,0.000543
Ni,0.603942,0.970406,7.461730,-311.347412,-13303.136642,2015.148481,-55.353609,6.122161e+07,-1.371471e+07,-2.345444,-0.415502,553.809000
Cu,1.415720,0.310762,6.709765,-1709.862433,-18334.188282,-2888.758001,-42.268511,8.938588e+07,1.540934e+07,-1.516821,0.129937,0.000000
constant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000


In [30]:
syen_n_data = df_syensqo.shape[0]
elements = ["Mn", "Co", "Ni", "Cu"]

param_scale = {'Mn':0.19, 'Co':5.6e-5, 'Ni':0.00014, 'Cu':0.000189}

m = pyo.ConcreteModel()

Mn_break_point = 10
Co_break_point = 20
Ni_break_point = 30
Cu_break_point = syen_n_data-1

m.elements = pyo.Set(initialize = elements)
m.syen_I = pyo.RangeSet(0, syen_n_data-1)

m.syen_I_Mn = pyo.RangeSet(0, Mn_break_point)
m.syen_I_Co = pyo.RangeSet(Mn_break_point+1, Co_break_point)
m.syen_I_Ni = pyo.RangeSet(Co_break_point+1, Ni_break_point)
m.syen_I_Cu = pyo.RangeSet(Ni_break_point+1, Cu_break_point)

m.syen_pH = pyo.Var(m.syen_I, domain=pyo.Reals, initialize=0.0)
m.syen_C_ext = pyo.Var(m.syen_I, domain=pyo.Reals, initialize=0.0)
m.syen_C_ext_net = pyo.Var(m.syen_I, domain=pyo.Reals, initialize=1e-3, bounds=(1e-7,0.01))
m.logsyen_C_ext_net = pyo.Var(m.syen_I, domain=pyo.Reals, initialize=0.5, bounds=(-7,2) )
m.syen_Cfeed = pyo.Var(m.syen_I, m.elements, domain=pyo.Reals, initialize=0.5)

m.syen_logD_Mn = pyo.Var(m.syen_I_Mn, domain=pyo.Reals, initialize=0.0)
m.syen_logD_Co = pyo.Var(m.syen_I_Co, domain=pyo.Reals, initialize=0.0)
m.syen_logD_Ni = pyo.Var(m.syen_I_Ni, domain=pyo.Reals, initialize=0.0)
m.syen_logD_Cu = pyo.Var(m.syen_I_Cu, domain=pyo.Reals, initialize=0.0)

m.syen_logD_pred = pyo.Var(m.syen_I, domain=pyo.Reals, initialize=0.0)

m.syen_alpha = pyo.Var(m.syen_I, m.elements, domain=pyo.Reals, initialize=0.0)
m.syen_beta = pyo.Var(m.syen_I, m.elements, domain=pyo.Reals, initialize=0.0)

for i in df_syensqo.index:
    m.syen_pH[i].fix(df_syensqo['pH'][i])
    m.syen_C_ext[i].fix(df_syensqo['[D2EHPA] '][i])
    for element in m.elements:
        m.syen_Cfeed[i, element].fix(df_syensqo[f'[{element}] (mol/L)'][i])

for i in m.syen_I_Mn:
    m.syen_logD_Mn[i].fix(df_syensqo['log D (obs)_Mn'][i])

for i in m.syen_I_Co:
    m.syen_logD_Co[i].fix(df_syensqo['log D (obs)_Co'][i])

for i in m.syen_I_Ni:
    m.syen_logD_Ni[i].fix(df_syensqo['log D (obs)_Ni'][i])

for i in m.syen_I_Cu:
    m.syen_logD_Cu[i].fix(df_syensqo['log D (obs)_Cu'][i])

@m.Constraint(m.syen_I, m.elements)
def syen_alpha_const(m, i, e):
    linear_term = df_da_silva_param['c1i'][e]*m.syen_Cfeed[i, "Mn"] + df_da_silva_param['c2i'][e]*m.syen_Cfeed[i, "Co"] +  df_da_silva_param['c3i'][e]*m.syen_Cfeed[i, "Ni"] + df_da_silva_param['c4i'][e]*m.syen_Cfeed[i, "Cu"]
    quad_term = df_da_silva_param['d3i'][e]*(m.syen_Cfeed[i, "Cu"])**2 + df_da_silva_param['d2i'][e]*(m.syen_Cfeed[i, "Ni"])**2 + df_da_silva_param['d1i'][e]*(m.syen_Cfeed[i, "Mn"])**2
    return (m.syen_alpha[i, e] - df_da_silva_param['a1i'][e]  - df_da_silva_param['a2i'][e]*m.syen_C_ext[i] 
            - linear_term 
            - quad_term 
            ) == 0

@m.Constraint(m.syen_I, m.elements)
def syen_b_const(m, i, e):
    return m.syen_beta[i, e] == df_da_silva_param['b1i'][e] + df_da_silva_param['b2i'][e]*m.logsyen_C_ext_net[i]

@m.Constraint(m.syen_I)
def syen_logCext_net_constraint(m, i):
    return 10**(m.logsyen_C_ext_net[i]) == (m.syen_C_ext_net[i])

@m.Constraint(m.syen_I)
def syen_Cext_net_constraint(m, i):
    return m.syen_C_ext_net[i] == m.syen_C_ext[i] - sum(df_da_silva_param['f'][e]*m.syen_Cfeed[i, e] for e in m.elements) - df_da_silva_param['f']['constant']

@m.Constraint(m.syen_I)
def syen_logD_pred_constraint(m, i):
    return m.syen_logD_pred[i] == sum((m.syen_alpha[i, e]*m.syen_pH[i] + m.syen_beta[i, e])*df_syensqo[f"{e}_el_fac"][i] for e in m.elements)

solver = pyo.SolverFactory('ipopt_v2')
solver.options['halt_on_ampl_error'] = 'yes'
results_syen = solver.solve(m, tee=True, symbolic_solver_labels=True)

     

RuntimeError: A feasible solution was not found, so no solution can be loaded.Please set opt.config.load_solutions=False to bypass this error.

In [ ]:
m.syen_alpha_const.pprint()

syen_alpha_const : Size=152, Index=syen_I*elements, Active=True
    Key        : Lower : Body                                                                                                                                                                                                                                                                                                                                                               : Upper : Active
     (0, 'Co') :   0.0 :          syen_alpha[0,Co] - 0.8954495449630535 - 0.2396028645041252*syen_C_ext[0] - (6.579919121114426*syen_Cfeed[0,Mn] - 2045.493340218392*syen_Cfeed[0,Co] - 16959.12315500826*syen_Cfeed[0,Ni] + 2403.701735542916*syen_Cfeed[0,Cu]) - (-11012933.49970208*syen_Cfeed[0,Cu]**2 + 82337296.79941548*syen_Cfeed[0,Ni]**2 - 43.51747358214003*syen_Cfeed[0,Mn]**2) :   0.0 :   True
     (0, 'Cu') :   0.0 :              syen_alpha[0,Cu] - 1.404779019960288 - 0.02579074217020462*syen_C_ext[0] - (6.41236937593556*syen_Cfeed[

In [ ]:
from idaes.core.util import DiagnosticsToolbox

dt = DiagnosticsToolbox(m)
dt.report_numerical_issues()

Model Statistics

    Jacobian Condition Number: 8.680E+06

------------------------------------------------------------------------------------
1 WARNINGS


------------------------------------------------------------------------------------
1 Cautions

    Caution: 456 Variables with value close to zero (tol=1.0E-08)

------------------------------------------------------------------------------------
Suggested next steps:

    display_constraints_with_large_residuals()
    compute_infeasibility_explanation()



In [ ]:
dt.display_constraints_with_large_residuals()

The following constraint(s) have large residuals (>1.0E-05):

    syen_alpha_const[0,Mn]: 1.29323E+00
    syen_alpha_const[0,Co]: 9.80857E-01
    syen_alpha_const[0,Ni]: 7.98162E-01
    syen_alpha_const[0,Cu]: 1.46762E+00
    syen_alpha_const[1,Mn]: 1.29323E+00
    syen_alpha_const[1,Co]: 9.80857E-01
    syen_alpha_const[1,Ni]: 7.98162E-01
    syen_alpha_const[1,Cu]: 1.46762E+00
    syen_alpha_const[2,Mn]: 1.29323E+00
    syen_alpha_const[2,Co]: 9.80857E-01
    syen_alpha_const[2,Ni]: 7.98162E-01
    syen_alpha_const[2,Cu]: 1.46762E+00
    syen_alpha_const[3,Mn]: 1.29323E+00
    syen_alpha_const[3,Co]: 9.80857E-01
    syen_alpha_const[3,Ni]: 7.98162E-01
    syen_alpha_const[3,Cu]: 1.46762E+00
    syen_alpha_const[4,Mn]: 1.29323E+00
    syen_alpha_const[4,Co]: 9.80857E-01
    syen_alpha_const[4,Ni]: 7.98162E-01
    syen_alpha_const[4,Cu]: 1.46762E+00
    syen_alpha_const[5,Mn]: 1.29323E+00
    syen_alpha_const[5,Co]: 9.80857E-01
    syen_alpha_const[5,Ni]: 7.98162E-01
    syen_alpha_con